In [3]:
import pandas as pd
import altair as alt

In [35]:
df = pd.read_csv("../data/filtered/province_data.csv")

In [36]:
df.head()

,REF_DATE,Gender,GEO,VALUE
0,2016,Women,"Canada, total",4277.0
1,2016,Men,"Canada, total",10114.0
2,2016,Women,Newfoundland and Labrador,12.0
3,2016,Men,Newfoundland and Labrador,46.0
4,2016,Women,Prince Edward Island,3.0


In [37]:
prov= "Newfoundland and Labrador"
filtered_df = df[(df["GEO"] == prov)]

In [38]:
filtered_df.head()

,REF_DATE,Gender,GEO,VALUE
2,2016,Women,Newfoundland and Labrador,12.0
3,2016,Men,Newfoundland and Labrador,46.0
30,2017,Women,Newfoundland and Labrador,20.0
31,2017,Men,Newfoundland and Labrador,50.0
58,2018,Women,Newfoundland and Labrador,18.0


In [12]:
selected_year = 2016

In [45]:
prov_ratio_values = filtered_df.apply(lambda row: row['VALUE'] / filtered_df.loc[(filtered_df['REF_DATE'] == row['REF_DATE']) & (filtered_df['Gender'] == 'Men'), 'VALUE'].iloc[0], axis=1)
prov_ratio = prov_ratio_values[prov_ratio_values != 1]
prov_ratio

2      0.260870
30     0.400000
58     0.391304
86     0.243243
114    0.310345
dtype: float64

In [28]:
canada_total = df[(df["GEO"] == "Canada, total")]
canada_total

,REF_DATE,Gender,GEO,VALUE
0,2016,Women,"Canada, total",4277.0
1,2016,Men,"Canada, total",10114.0
28,2017,Women,"Canada, total",4137.0
29,2017,Men,"Canada, total",9754.0
56,2018,Women,"Canada, total",3786.0
57,2018,Men,"Canada, total",8747.0
84,2019,Women,"Canada, total",2688.0
85,2019,Men,"Canada, total",6204.0
112,2020,Women,"Canada, total",2809.0
113,2020,Men,"Canada, total",6140.0


In [30]:
canada_total_ratio = canada_total.pivot(index='REF_DATE', columns='Gender', values='VALUE')
canada_total_ratio['Ratio'] = canada_total_ratio['Women'] / canada_total_ratio['Men']

canada_total_ratio = canada_total_ratio.reset_index()[['REF_DATE', 'Ratio']]
canada_total_ratio.columns = ['Year', 'Ratio']


   Year     Ratio
0  2016  0.422879
1  2017  0.424134
2  2018  0.432834
3  2019  0.433269
4  2020  0.457492


In [24]:
filtered_df

,REF_DATE,Gender,GEO,VALUE,Ratio
2,2016,Women,Newfoundland and Labrador,12.0,0.260870
30,2017,Women,Newfoundland and Labrador,20.0,0.400000
58,2018,Women,Newfoundland and Labrador,18.0,0.391304
86,2019,Women,Newfoundland and Labrador,9.0,0.243243
114,2020,Women,Newfoundland and Labrador,9.0,0.310345


In [31]:

chart = alt.Chart(filtered_df).mark_line().encode(
        x=alt.X('REF_DATE:O', axis=alt.Axis(title='Year')),
        y=alt.Y('Ratio:Q', axis=alt.Axis(title='Number of People')),
        color='GEO:N',
        tooltip=['GEO:N', 'Ratio:Q']
    ).properties(
        title='Number of Men and Women in Executive Positions in {} Over the Years'.format(prov),
        width=600,
        height=200
    )

if selected_year is not None:
    rule = alt.Chart(pd.DataFrame({'selected_year': [selected_year]})).mark_rule(color='red').encode(
        x='selected_year:O'
    ).transform_filter(
        alt.FieldEqualPredicate(field='selected_year', equal=selected_year)
    )
    chart_with_marker = chart + rule
else:
    chart_with_marker = chart

canada_total_ratio = alt.Chart(canada_total_ratio).mark_line(strokeDash=[1,1]).encode(
    x='Year:O',
    y='Ratio:Q'
)

chart_with_marker = chart_with_marker + canada_total_ratio

In [61]:
filtered_df = df[(df["GEO"] == prov)]

canada_total = {
'Year': [2016, 2017, 2018, 2019, 2020],
'Ratio': [0.422879, 0.424134, 0.432834, 0.433269, 0.457492]
}
canada_total_ratio = pd.DataFrame(canada_total)

prov_ratio_values = filtered_df.apply(lambda row: row['VALUE'] / filtered_df.loc[(filtered_df['REF_DATE'] == row['REF_DATE']) & (filtered_df['Gender'] == 'Men'), 'VALUE'].iloc[0], axis=1)
prov_ratio = prov_ratio_values[prov_ratio_values != 1]

province_gender_ratio = pd.DataFrame({
'Year': range(2016, 2021),
'Ratio': prov_ratio
})

chart = alt.Chart(province_gender_ratio).mark_line().encode(
    x=alt.X('Year:O', axis=alt.Axis(title='Year')),
    y=alt.Y('Ratio:Q', axis=alt.Axis(title='Number of People')),
    color = alt.value("#228B22")
).properties(
    title='Number of Men and Women in Executive Positions in {} Over the Years'.format(prov),
    width=1200,
    height=200
)

if selected_year is not None:
    rule = alt.Chart(pd.DataFrame({'selected_year': [selected_year]})).mark_rule(color='gray').encode(
        x='selected_year:O'
    ).transform_filter(
        alt.FieldEqualPredicate(field='selected_year', equal=selected_year)
    )
    chart_with_marker = chart + rule
else:
    chart_with_marker = chart

canada_tot_ratio = alt.Chart(canada_total_ratio).mark_line(strokeDash=[1,1]).encode(
    x='Year:O',
    y='Ratio:Q',
    color = alt.value("#800080")
)

chart_with_marker = chart_with_marker + canada_tot_ratio

chart_with_marker

alt.LayerChart(...)

In [48]:
chart_with_marker

alt.LayerChart(...)

1. remove space before 2016 and after 2020
2. static line showing national average
3. line showing ratio by province